# Download GSE's

## Libraries

In [1]:
import warnings

import urllib.request
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

import pandas as pd
import matplotlib.pyplot as plt
import numpy

import GEOparse

%load_ext rpy2.ipython

## Functions

In [22]:
def search_str_index(s_list:list, search_str:str) -> list:
    """
    Get the indexes in a list that contain a `search_str`
    
    Parameters:
        s_list (list): the list to search in
        search_str (str): the str to search for, it only must be contained

    Returns:
        matched_indexes (list): A list of the indexes where it was found
    """
    
    matched_indexes = []
    i = 0
    length = len(s_list)

    while i < length:
        if type(s_list[i]) is not str:
            i += 1
            continue
        if search_str in s_list[i]:
            matched_indexes.append(i)
        i += 1
        
    return matched_indexes
    


def get_geo_exprs(gse_str='', data_dir='/root/datos/maestria/netopaas/lung_scRNA'):
    """
    Builds a metada matrix from geo code then matrix SOFT file for scRNA-seq data that
    doesn't have the expression matrix attached. A csv is saved in
    f'{data_dir}/{gse_str}/{gse_str}_metadata.csv'
    
    Parameters:
        gse_str (str): The string of the GSE to be gotten
        data_dir (str): Where the CSV is to be saved

    Returns:
        metadata (dict): A dict with first level the features, second the sample
    
    """
    gse = GEOparse.get_GEO(geo=gse_str, destdir=f"{data_dir}/{gse_str}/", silent=True, include_data=True)

    # Get expression data and metadata matrices
    exprs = []
    gsmNames = []
    metadata = {}
    sup_dict = {}
    for gsm_name, gsm in gse.gsms.items():
        if gsm.metadata['type'][0]=='SRA':
             # Expression data
            # print(gsm.__dict__)
            if len(gsm.table)>0:
                # TODO will there really be a table here anytime?
                # if so run code here
                pass
            else:
                # Get the supplementary files with their type because no table is attached
                sup_file_url = gsm.metadata['supplementary_file_1'][0]
                
                # TODO it is in this array but no standard index, search for it later
                l1 = gsm.metadata['data_processing']
                s = 'upplementary'
                matched_indexes = search_str_index(l1, s)
                sup_file_type = l1[matched_indexes[0]].split(':')[1]
                
                l1 = gsm.metadata['data_processing']
                s = 'Genome_build'
                matched_indexes = search_str_index(l1, s)
                genome_build = l1[matched_indexes[0]].split(':')[1]
                
                if not 'sup_file_url' in sup_dict:
                    sup_dict['sup_file_url'] = {}
                sup_dict['sup_file_url'][gsm_name] = sup_file_url
                
                if not 'sup_file_type' in sup_dict:
                    sup_dict['sup_file_type'] = {}
                sup_dict['sup_file_type'][gsm_name] = sup_file_type
                
                if not 'genome_build' in sup_dict:
                    sup_dict['genome_build'] = {}
                sup_dict['genome_build'][gsm_name] = genome_build
                
                
                # print('No expression table, saving supplementary file url'
                #              f'{sup_file_url} with type: {sup_file_type}')
            if hasattr(gsm, 'SRA'):
                warnings.warn("There is an SRArun access table, consider using "
                              "your snakemake workflow to parallely download them")
                
    # Metadata
            for key,value in gsm.metadata.items():
                if (key=='characteristics_ch1' or key=='characteristics_ch2') and (len([i for i in value if i!=''])>1 or value[0].find(': ')!=-1):
                    tmpVal = 0
                    for tmp in value:
                        splitUp = [i.strip() for i in tmp.split(':')]
                        if len(splitUp)==2:
                            if not splitUp[0] in metadata:
                                metadata[splitUp[0]] = {}
                            metadata[splitUp[0]][gsm_name] = splitUp[1]
                        else:
                            if not key in metadata:
                                metadata[key] = {}
                            metadata[key][gsm_name] = splitUp[0]
                else:
                    if not key in metadata:
                        metadata[key] = {}
                    if len(value)==1:
                        metadata[key][gsm_name] = ' '.join([j.replace(',',' ') for j in value])
            ftp_name = sup_dict['sup_file_url'][gsm_name].split('/')[-1]
            
            key = 'sup_type'
            if not key in metadata:
                metadata[key] = {}
            metadata[key][gsm_name] = sup_dict['sup_file_type'][gsm_name]
            
            key = 'local_path'
            if not key in metadata:
                metadata[key] = {}
            metadata[key][gsm_name] = f'{data_dir}/{gse_str}/{ftp_name}'
            
            key = 'genome_build'
            if not key in metadata:
                metadata[key] = {}
            metadata[key][gsm_name] = sup_dict['genome_build'][gsm_name]
            
            metadata['local_path'][gsm_name] = f'{data_dir}/{gse_str}/{ftp_name}'
    pd.DataFrame(metadata).to_csv(f'{data_dir}/{gse_str}/{gse_str}_metadata.csv')

    return metadata

def download_url(args, data_dir='/root/datos/maestria/netopaas/lung_scRNA'):
    t0 = time.time()
    #Extract url and file path from args
    url = args[0]
    path = args[1]
    try:
        # For getting ftp that does not need 
        gsm_path, response = urllib.request.urlretrieve(url,
                                      path)
        return(url, time.time() - t0)
    except Exception as e:
        print('Exception in download_url():', e)


def download_parallel(args):
    """
    Downloads the zipped array of urls  in 1st pos with paths in 2nd pos
    
    """
    cpus = int(cpu_count()/3)
    print("CPUS: ", cpus)
    
    with ThreadPool(cpus -1 ) as pool:
        for result in pool.imap_unordered(download_url, args):
            print('url:', result[0], 'time (s):', result[1])


In [23]:
download_parallel([('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE136nnn/GSE136831/suppl/GSE136831%5FRawCounts%5FSparse%2Emtx%2Egz', '/root/datos/maestria/netopaas/lung_scRNA/GSE136831/sparse.mtx.gz'),
                  ('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE136nnn/GSE136831/suppl/GSE136831%5FAllCells%2EcellBarcodes%2Etxt%2Egz','barcodes.txt.gz'),
                  ('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE136nnn/GSE136831/suppl/GSE136831%5FAllCells%2EGeneIDs%2Etxt%2Egz','ids.txt.gz')])

CPUS:  26
url: https://ftp.ncbi.nlm.nih.gov/geo/series/GSE136nnn/GSE136831/suppl/GSE136831%5FAllCells%2EGeneIDs%2Etxt%2Egz time (s): 0.6091160774230957
url: https://ftp.ncbi.nlm.nih.gov/geo/series/GSE136nnn/GSE136831/suppl/GSE136831%5FAllCells%2EcellBarcodes%2Etxt%2Egz time (s): 0.7396457195281982
url: https://ftp.ncbi.nlm.nih.gov/geo/series/GSE136nnn/GSE136831/suppl/GSE136831%5FRawCounts%5FSparse%2Emtx%2Egz time (s): 113.40795230865479


## Data

In [177]:
gse_IV= 'GSE123902' # lung metastasis 7 actual tumor
meta_IV = get_geo_exprs(gse_IV)

urls = list(meta_IV['supplementary_file_1'].values())
paths = list(meta_IV['local_path'].values())
args_IV = zip(urls, paths)

In [183]:
meta_IV['geo_accession']

{'GSM3516662': 'GSM3516662',
 'GSM3516663': 'GSM3516663',
 'GSM3516664': 'GSM3516664',
 'GSM3516665': 'GSM3516665',
 'GSM3516666': 'GSM3516666',
 'GSM3516667': 'GSM3516667',
 'GSM3516668': 'GSM3516668',
 'GSM3516669': 'GSM3516669',
 'GSM3516670': 'GSM3516670',
 'GSM3516671': 'GSM3516671',
 'GSM3516672': 'GSM3516672',
 'GSM3516673': 'GSM3516673',
 'GSM3516674': 'GSM3516674',
 'GSM3516675': 'GSM3516675',
 'GSM3516676': 'GSM3516676',
 'GSM3516677': 'GSM3516677',
 'GSM3516678': 'GSM3516678'}

In [115]:
download_parallel(args_IV, gse_IV)

CPUS:  26
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516662/suppl/GSM3516662_MSK_LX653_PRIMARY_TUMOUR_dense.csv.gz time (s): 1.8080072402954102
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516670/suppl/GSM3516670_MSK_LX680_PRIMARY_TUMOUR_dense.csv.gz time (s): 2.0990450382232666
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516664/suppl/GSM3516664_MSK_LX666_METASTASIS_dense.csv.gz time (s): 2.1689655780792236
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516675/suppl/GSM3516675_MSK_LX684_NORMAL_dense.csv.gz time (s): 2.200122833251953
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516676/suppl/GSM3516676_MSK_LX685_NORMAL_dense.csv.gz time (s): 2.2731893062591553
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516669/suppl/GSM3516669_MSK_LX679_PRIMARY_TUMOUR_dense.csv.gz time (s): 2.4018094539642334
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516674/suppl/GSM3516674_MSK_LX684_PRIMARY_TUMOUR_den

In [17]:
gse_both= 'GSE136831' # lung metastasis 7 actual tumor
meta_both = get_geo_exprs(gse_both)

urls = list(meta_both['supplementary_file_1'].values())
paths = list(meta_both['local_path'].values())
args_IV = zip(urls, paths)

In [15]:
list(meta_both['supplementary_file_1'].values())

['ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GSM4058900/suppl/GSM4058900_133C-a.dgecounts.rds.gz',
 'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GSM4058901/suppl/GSM4058901_137C-a.dgecounts.rds.gz',
 'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GSM4058902/suppl/GSM4058902_034C.dgecounts.rds.gz',
 'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GSM4058903/suppl/GSM4058903_218C-a.dgecounts.rds.gz',
 'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GSM4058904/suppl/GSM4058904_226C-a.dgecounts.rds.gz',
 'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GSM4058905/suppl/GSM4058905_244C.dgecounts.rds.gz',
 'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GSM4058906/suppl/GSM4058906_098C-a.dgecounts.rds.gz',
 'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GSM4058907/suppl/GSM4058907_465C.dgecounts.rds.gz',
 'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GSM4058908/suppl/GSM4058908_396C.dgecounts.rds.gz',
 'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4058nnn/GS

In [183]:
meta_IV['geo_accession']

{'GSM3516662': 'GSM3516662',
 'GSM3516663': 'GSM3516663',
 'GSM3516664': 'GSM3516664',
 'GSM3516665': 'GSM3516665',
 'GSM3516666': 'GSM3516666',
 'GSM3516667': 'GSM3516667',
 'GSM3516668': 'GSM3516668',
 'GSM3516669': 'GSM3516669',
 'GSM3516670': 'GSM3516670',
 'GSM3516671': 'GSM3516671',
 'GSM3516672': 'GSM3516672',
 'GSM3516673': 'GSM3516673',
 'GSM3516674': 'GSM3516674',
 'GSM3516675': 'GSM3516675',
 'GSM3516676': 'GSM3516676',
 'GSM3516677': 'GSM3516677',
 'GSM3516678': 'GSM3516678'}

In [115]:
download_parallel(args_IV, gse_IV)

CPUS:  26
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516662/suppl/GSM3516662_MSK_LX653_PRIMARY_TUMOUR_dense.csv.gz time (s): 1.8080072402954102
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516670/suppl/GSM3516670_MSK_LX680_PRIMARY_TUMOUR_dense.csv.gz time (s): 2.0990450382232666
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516664/suppl/GSM3516664_MSK_LX666_METASTASIS_dense.csv.gz time (s): 2.1689655780792236
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516675/suppl/GSM3516675_MSK_LX684_NORMAL_dense.csv.gz time (s): 2.200122833251953
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516676/suppl/GSM3516676_MSK_LX685_NORMAL_dense.csv.gz time (s): 2.2731893062591553
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516669/suppl/GSM3516669_MSK_LX679_PRIMARY_TUMOUR_dense.csv.gz time (s): 2.4018094539642334
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3516nnn/GSM3516674/suppl/GSM3516674_MSK_LX684_PRIMARY_TUMOUR_den

### Handling gene annotations (some tests)

In [46]:
gsm = 'GSM3516662_MSK_LX653_PRIMARY_TUMOUR_dense.csv.gz'
adata_IV = pd.read_csv(f'{data_dir}/{gse_IV}/{gsm}')

In [124]:
adata_IV.index =  adata_IV['Unnamed: 0']
adata_IV = adata_IV.drop(columns='Unnamed: 0')
adata_IV

,TSPAN6,DPM1,SCYL3,C1ORF112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,RP11-298A10.1,RP11-180P8.1,RP11-9J18.1,RP11-151G14.2,RP11-1037J10.1,EXOC3L2,RP11-57A19.7,RP11-419I17.1,AC013271.5,RP11-122G18.12
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
120703436614579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120726924712805,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120772934388134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120786758522667,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120797912811867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241057699838877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241057699907941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241114561854197,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [181]:
gse_I = 'GSE148071' # chinese no raw files
meta_I = get_geo_exprs(gse_I)

urls = list(meta_I['supplementary_file_1'].values())
paths = list(meta_I['local_path'].values())
args_I = zip(urls, paths)

In [117]:
download_parallel(args_I, gse_I)

CPUS:  26
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4453nnn/GSM4453599/suppl/GSM4453599_P24_exp.txt.gz time (s): 1.2470176219940186
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4453nnn/GSM4453577/suppl/GSM4453577_P2_exp.txt.gz time (s): 1.3342995643615723
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4453nnn/GSM4453587/suppl/GSM4453587_P12_exp.txt.gz time (s): 1.6076200008392334
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4453nnn/GSM4453597/suppl/GSM4453597_P22_exp.txt.gz time (s): 1.6947636604309082
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4453nnn/GSM4453594/suppl/GSM4453594_P19_exp.txt.gz time (s): 1.9105088710784912
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4453nnn/GSM4453595/suppl/GSM4453595_P20_exp.txt.gz time (s): 1.9886648654937744
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4453nnn/GSM4453583/suppl/GSM4453583_P8_exp.txt.gz time (s): 2.0728156566619873
url: ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4453nnn/GSM4453601/suppl/GSM4453601_P26_exp.txt.gz time 

In [207]:
aro = ['ENSG00000125085','ENSG00000125085']

In [197]:
%%R -i aro
# biomaRt::listEnsemblArchives()
library(biomaRt)
ensembl <- useEnsembl(biomart = "genes", dataset = "hsapiens_gene_ensembl")

R[write to console]: Error in .processResults(postRes, mart = mart, hostURLsep = sep, fullXmlQuery = fullXmlQuery,  : 
  Query ERROR: caught BioMart::Exception::Usage: Filter NA NOT FOUND




Error in .processResults(postRes, mart = mart, hostURLsep = sep, fullXmlQuery = fullXmlQuery,  : 
  Query ERROR: caught BioMart::Exception::Usage: Filter NA NOT FOUND


In [208]:
%%R -i aro
aro = unlist(aro)
ensembl.ann = getBM(attributes = c("ensembl_gene_id",
                                            "percentage_gene_gc_content",
                                            "gene_biotype",
                                            "chromosome_name",
                                            "external_gene_name"),
                             filters = "ensembl_gene_id",
                             values = aro,
                             mart = ensembl)


In [209]:
%R print(ensembl.ann)

[1] ensembl_gene_id            percentage_gene_gc_content
[3] gene_biotype               chromosome_name           
[5] external_gene_name        
<0 rows> (or 0-length row.names)


,ensembl_gene_id,percentage_gene_gc_content,gene_biotype,chromosome_name,external_gene_name


In [122]:
gsm = 'GSM4453580_P5_exp.txt.gz'
adata_I = pd.read_table(f'{data_dir}/{gse_I}/{gsm}')
adata_I = adata_I.transpose()

In [123]:
adata_I

,PWP1,FTH1,PERP,RPL37,MT-RNR2,AKR1C3,ATP6V0E1,TPT1,RPL13A,VRK2,...,LINC02267,AC109466.1,LINC01938,AC008708.2,AL360175.1,AL121718.1,C7orf71,YWHAEP1,LINC01287,LUZP4
8_CGCAATGCATCG,2,0,4,66,2006,0,7,25,36,1,...,0,0,0,0,0,0,0,0,0,0
8_GCTCTATAAGTA,2,1,7,36,1867,0,5,9,27,1,...,0,0,0,0,0,0,0,0,0,0
8_GCGCGGACCAGG,2,1,0,50,1509,2,3,13,46,0,...,0,0,0,0,0,0,0,0,0,0
8_CGGTGGCCCGTG,1,9,1,28,1732,1,2,17,19,0,...,0,0,0,0,0,0,0,0,0,0
8_TTCCGCATCTTA,0,0,1,10,2109,0,4,14,17,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9_CGTCTGCTCCAG,0,0,0,1,339,0,0,3,2,0,...,0,0,0,0,0,0,0,0,0,0
9_TAACGGTCTATT,0,6,0,2,464,0,2,6,0,0,...,0,0,0,0,0,0,0,0,0,0
9_TCAATGTGGGGT,0,14,0,5,353,0,0,4,3,0,...,0,0,0,0,0,0,0,0,0,0
9_CAGCACTGCTCT,0,0,0,2,369,0,3,1,1,0,...,0,0,0,0,0,0,0,0,0,0
